In [1]:
from data_reading import *

In [26]:
lep_jou_aff = lep_article_data['Journal'].value_counts().reset_index()
lep_jou_aff.columns = ['Journal', 'J_Count']

lep_ja_sum = lep_jou_aff['J_Count'].sum()

lep_jou_aff['J_Share (%)'] = round((lep_jou_aff['J_Count'] / lep_ja_sum) * 100, 2)

lep_ja_top20 = lep_jou_aff.iloc[:20,:]

ipd.display(lep_ja_top20)

lep_ja_top20_bar = px.bar(lep_ja_top20,
                    x = 'Journal',
                    y = 'J_Count',
                    color = 'J_Share (%)',
                    title = 'Top Contributing Journals')
lep_ja_top20_bar.update_layout(title_x = 0.5, showlegend=True, height = 1000, width = 1300)

lep_ja_top20_bar.show()

,Journal,J_Count,J_Share (%)
0,PLoS neglected tropical diseases,151,4.46
1,The American journal of tropical medicine and ...,129,3.81
2,PloS one,101,2.98
3,BMC infectious diseases,54,1.59
4,Acta tropica,53,1.56
5,Epidemiology and infection,44,1.30
6,Infection and immunity,41,1.21
7,Tropical animal health and production,39,1.15
8,"Comparative immunology, microbiology and infec...",39,1.15
9,The Southeast Asian journal of tropical medici...,36,1.06


In [16]:
lep_author_data['A_Fullname'] = lep_author_data['AuthorLastname'] + ' ' + lep_author_data['AuthorInitials']

lep_id_jou = lep_article_data[['PMID', 'Journal']]
lep_id_auth = lep_author_data[['PMID', 'A_Fullname']]
lep_id_aff = lep_article_data[['PMID', 'FirstAuthorAffiliation']]

relation_list = [lep_id_jou, lep_id_aff]

for relation in relation_list:
    merge = pd.merge(relation, lep_id_auth, on = 'PMID', how = 'outer')
    total = merge.groupby([relation.columns[1], lep_id_auth.columns[1]]).size().reset_index(name = 'Count')
    maximum = total.groupby(relation.columns[1])['Count'].transform(lambda x: x == x.max())
    auth_max = total.loc[maximum]
    auth_max.columns = [relation.columns[1], 'Author_Fullname', 'Author_Count']
    order = auth_max.sort_values(by = 'Author_Count',
                                        ascending = False,
                                        ignore_index = True)
    topN_auth = order.iloc[:50,:]
    #ipd.display(topN_auth)
    topN_bar = px.bar(topN_auth,
                    x = 'Author_Fullname',
                    y = 'Author_Count',
                    color = relation.columns[1],
                    title = f'{relation.columns[1]} Top Contributing Authors')
    topN_bar.update_layout(title_x = 0.5, showlegend=True, height = 1000, width = 2000)
    topN_bar.update_traces(width=1)
    topN_bar.update_xaxes(tickangle=45)
    topN_bar.show()